In [25]:
import numpy as np
import cv2

In [26]:
cap = cv2.VideoCapture("../images/production ID_4152552.mp4")
n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(n_frames)

w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

fourcc = cv2.VideoWriter_fourcc(*'MJPG')
out = cv2.VideoWriter('../images/stabilized_video_out.mp4', fourcc, fps, (w, h))

528


In [27]:
_, prev = cap.read()
prev_gray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)

In [28]:
transforms = np.zeros((n_frames-1,3), np.float32)
for i in range(n_frames-2):
    prev_pts = cv2.goodFeaturesToTrack(prev_gray, maxCorners=200,
                                     qualityLevel=0.01,
                                     minDistance=30,
                                     blockSize=3)
    print(len(prev_pts))
    success, curr = cap.read()
    if not success:
        break
        
    curr_gray = cv2.cvtColor(curr, cv2.COLOR_BGR2GRAY)
    curr_pts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, curr_gray, prev_pts, None)
    
    assert prev_pts.shape == curr_pts.shape
    
    idx = np.where(status==1)[0]
    prev_pts = prev_pts[idx]
    curr_pts = curr_pts[idx]
    
    #Find transformation matrix
    m = cv2.estimateAffinePartial2D(prev_pts, curr_pts)
    
    dx = m[0][0,2]
    dy = m[0][1,2]
    
    da = np.arctan2(m[0][1,0], m[0][0,0])
     # Store transformation
    transforms[i] = [dx,dy,da]
    
  # Move to next frame
    prev_gray = curr_gray
 
    print("Frame: " + str(i) +  "/" + str(n_frames) + " -  Tracked points : " + str(len(prev_pts)))


200
Frame: 0/528 -  Tracked points : 200
200
Frame: 1/528 -  Tracked points : 200
200
Frame: 2/528 -  Tracked points : 200
200
Frame: 3/528 -  Tracked points : 200
200
Frame: 4/528 -  Tracked points : 200
200
Frame: 5/528 -  Tracked points : 200
200
Frame: 6/528 -  Tracked points : 200
200
Frame: 7/528 -  Tracked points : 200
200
Frame: 8/528 -  Tracked points : 200
200
Frame: 9/528 -  Tracked points : 200
200
Frame: 10/528 -  Tracked points : 200
200
Frame: 11/528 -  Tracked points : 200
200
Frame: 12/528 -  Tracked points : 200
200
Frame: 13/528 -  Tracked points : 200
200
Frame: 14/528 -  Tracked points : 199
200
Frame: 15/528 -  Tracked points : 200
200
Frame: 16/528 -  Tracked points : 200
200
Frame: 17/528 -  Tracked points : 200
200
Frame: 18/528 -  Tracked points : 200
200
Frame: 19/528 -  Tracked points : 200
200
Frame: 20/528 -  Tracked points : 200
200
Frame: 21/528 -  Tracked points : 200
200
Frame: 22/528 -  Tracked points : 200
200
Frame: 23/528 -  Tracked points : 200
20

Frame: 196/528 -  Tracked points : 200
200
Frame: 197/528 -  Tracked points : 200
200
Frame: 198/528 -  Tracked points : 200
200
Frame: 199/528 -  Tracked points : 200
200
Frame: 200/528 -  Tracked points : 200
200
Frame: 201/528 -  Tracked points : 200
200
Frame: 202/528 -  Tracked points : 200
200
Frame: 203/528 -  Tracked points : 200
200
Frame: 204/528 -  Tracked points : 200
200
Frame: 205/528 -  Tracked points : 200
200
Frame: 206/528 -  Tracked points : 200
200
Frame: 207/528 -  Tracked points : 200
200
Frame: 208/528 -  Tracked points : 200
200
Frame: 209/528 -  Tracked points : 200
200
Frame: 210/528 -  Tracked points : 200
200
Frame: 211/528 -  Tracked points : 200
200
Frame: 212/528 -  Tracked points : 200
200
Frame: 213/528 -  Tracked points : 200
200
Frame: 214/528 -  Tracked points : 200
200
Frame: 215/528 -  Tracked points : 200
200
Frame: 216/528 -  Tracked points : 200
200
Frame: 217/528 -  Tracked points : 200
200
Frame: 218/528 -  Tracked points : 200
200
Frame: 219/

200
Frame: 390/528 -  Tracked points : 199
200
Frame: 391/528 -  Tracked points : 199
200
Frame: 392/528 -  Tracked points : 198
200
Frame: 393/528 -  Tracked points : 198
200
Frame: 394/528 -  Tracked points : 200
200
Frame: 395/528 -  Tracked points : 200
200
Frame: 396/528 -  Tracked points : 200
200
Frame: 397/528 -  Tracked points : 200
200
Frame: 398/528 -  Tracked points : 199
200
Frame: 399/528 -  Tracked points : 200
200
Frame: 400/528 -  Tracked points : 199
200
Frame: 401/528 -  Tracked points : 200
200
Frame: 402/528 -  Tracked points : 200
200
Frame: 403/528 -  Tracked points : 197
200
Frame: 404/528 -  Tracked points : 194
200
Frame: 405/528 -  Tracked points : 198
200
Frame: 406/528 -  Tracked points : 200
200
Frame: 407/528 -  Tracked points : 200
200
Frame: 408/528 -  Tracked points : 200
200
Frame: 409/528 -  Tracked points : 200
200
Frame: 410/528 -  Tracked points : 199
200
Frame: 411/528 -  Tracked points : 200
200
Frame: 412/528 -  Tracked points : 200
200
Frame: 

In [35]:
trajectory = np.cumsum(transforms, axis=0)
SMOOTHING_RADIUS = 10

In [30]:
def movingAverage(curve, radius):
    window_size = 2*radius + 1
    f = np.ones(window_size)/window_size
    curve_pad = np.lib.pad(curve, (radius, radius), 'edge')
    curve_smoothed = np.convolve(curve_pad, f, mode='same')
    curve_smoothed = curve_smoothed[radius:-radius]
    return curve_smoothed

In [31]:
def smooth(trajectory):
    smoothed_trajectory = np.copy(trajectory)
    for i in range(3):
        smoothed_trajectory[:,i] = movingAverage(trajectory[:,i], radius = SMOOTHING_RADIUS)
    return smoothed_trajectory

In [32]:
smoothed_trajectory = smooth(trajectory)
# Calculate difference in smoothed_trajectory and trajectory
difference = smoothed_trajectory - trajectory
 
# Calculate newer transformation array
transforms_smooth = transforms + difference

In [33]:
def fixBorder(frame):
  s = frame.shape
  # Scale the image 4% without moving the center
  T = cv2.getRotationMatrix2D((s[1]/2, s[0]/2), 0, 1.04)
  frame = cv2.warpAffine(frame, T, (s[1], s[0]))
  return frame

In [34]:
# Reset stream to first frame
cap.set(cv2.CAP_PROP_POS_FRAMES, 0) 
 
# Write n_frames-1 transformed frames
for i in range(n_frames-2):
  # Read next frame
  success, frame = cap.read()
  if not success:
    break
 
  # Extract transformations from the new transformation array
  dx = transforms_smooth[i,0]
  dy = transforms_smooth[i,1]
  da = transforms_smooth[i,2]
 
  # Reconstruct transformation matrix accordingly to new values
  m = np.zeros((2,3), np.float32)
  m[0,0] = np.cos(da)
  m[0,1] = -np.sin(da)
  m[1,0] = np.sin(da)
  m[1,1] = np.cos(da)
  m[0,2] = dx
  m[1,2] = dy
 
  # Apply affine wrapping to the given frame
  frame_stabilized = cv2.warpAffine(frame, m, (w,h))
 
  # Fix border artifacts
  frame_stabilized = fixBorder(frame_stabilized) 
 
  # Write the frame to the file
  frame_out = cv2.hconcat([frame, frame_stabilized])
 
  # If the image is too big, resize it.
  if(frame_out.shape[1] == 1920):
    frame_out = cv2.resize(frame_out, (frame_out.shape[1]/2, frame_out.shape[0]/2));
  cv2.imshow("Before and After", frame_out)
  cv2.waitKey(10)
  out.write(frame_out)
    
cap.release()
cv2.destroyAllWindows()